# Embeddings and Sentence Classification

<center>
    <img src="embeddings.jpeg">
</center>

### Introduction

Embeddings are a way to represent words (or more generally, *tokens*) as vectors. These vectors are useful for many tasks in NLP, including but not limited to: Text Generation, Machine Translation, and Sentence Classification. In this notebook, we will be exploring the concept of Embeddings, and using them for Sentence Classification.

In [1]:
import re

import numpy as np
import pandas as pd

import nltk
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split

import gensim.downloader as api
from gensim.models.word2vec import Word2Vec

from gpt4all import Embed4All

import torch
import torch.nn as nn

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Exploring Embeddings

Put simply, Embeddings are fixed-size **dense** vector representations of tokens in natural language. This means you can represent words as vectors, sentences as vectors, even other entities like entire graphs as vectors.

So what really makes them different from something like One-Hot vectors?

What's special is that they have semantic meaning baked into them. This means you can model relationships between entities in text, which itself leads to a lot of fun applications. All modern architectures make use of Embeddings in some way.

You can read more about them [here](https://aman.ai/primers/ai/word-vectors/).

We will be using *pretrained* Embeddings: this means that we will be using Embeddings that have already been trained on a large corpus of text. This is because training Embeddings from scratch is a very computationally expensive task, and we don't have the resources to do so. Fortunately, there were some good samaritans who have already done this for us, and we can use their publicly available Embeddings for our own tasks.


This part will allow us to explore what Embeddings are. We will load in pretrained Embeddings here and examine some of their properties. If you're interested, feel free look up the [Word2Vec model](https://arxiv.org/abs/1301.3781): this is the model that was trained to give us the embeddings you will see below.

In [2]:
# Download the pretrained word2vec model (this may take a few minutes)
corpus = api.load('text8')
w2vmodel = Word2Vec(corpus)

print("Done loading word2vec model!")

Done loading word2vec model!


Now that we've loaded in the Embeddings, we can create an Embedding **layer** in PyTorch, `nn.Embedding`, that will perform the processing step for us.

Note in the following cell how there is a given **vocab size** and **embedding dimension** for the model: this is important to note because some sets of Embeddings may be defined for a large set of words (a large vocab), whereas older ones perhaps have a smaller set (a small vocab); the Embedding dimension essentially tells us how many *features* have been learned for a given word, that will allow us to perform further processing on top of.

In [3]:
# Define embedding layer using gensim
embedding_layer = nn.Embedding.from_pretrained(torch.FloatTensor(w2vmodel.wv.vectors))

# Get some information from the w2vmodel
print(f"Vocab size: {len(w2vmodel.wv.key_to_index)}")

print(f"Some of the words in the vocabulary:\n{list(w2vmodel.wv.key_to_index.keys())[:10]}")

print(f"Embedding dimension: {w2vmodel.wv.vectors.shape[1]}")

Vocab size: 71290
Some of the words in the vocabulary:
['the', 'of', 'and', 'one', 'in', 'a', 'to', 'zero', 'nine', 'two']
Embedding dimension: 100


Now, for a demonstration, we instantiate two words, turn them into numbers (encoding them via their index in the vocab), and pass them through the Embedding layer. 

Note how the resultant Embeddings both have the same shape: 1 word, and 100 elements in the vector.

In [4]:
# Take two words and get their embeddings
word1 = "king"
word2 = "queen"

def word2vec(word):
    return embedding_layer(torch.LongTensor([w2vmodel.wv.key_to_index[word]]))

king_embedding = word2vec(word1)
queen_embedding = word2vec(word2)

print(f"Embedding Shape for '{word1}': {king_embedding.shape}")
print(f"Embedding Shape for '{word2}': {queen_embedding.shape}")

Embedding Shape for 'king': torch.Size([1, 100])
Embedding Shape for 'queen': torch.Size([1, 100])


When we have vectors whose scale is arbitrary, one nice way to measure how *similar* they are is with the Cosine Similarity measure.


$$ \text{Cosine Similarity}(\mathbf{u},\mathbf{v}) = \frac{\mathbf{u} \cdot \mathbf{v}}{\|\mathbf{u}\| \|\mathbf{v}\|} $$


We can apply this idea to our Embeddings. To see how "similar" two words are to the model, we can generate their Embeddings and take the Cosine Similarity of them. This will be a number between -1 and 1 (just like the range of the cosine function). When the number is close to 0, the words are not similar.

In [ ]:
def cosine_similarity(vec1, vec2):
    '''
    Computes the cosine similarity between two vectors
    '''

    # Compute the cosine similarity between the two vectors (using PyTorch)
    dotproduct = np.dot(vec1,vec2)
    vec1_norm = torch.norm(vec1)
    vec2_norm = torch.norm(vec2)
    cos_sim = dotproduct/(vec1_norm*vec2_norm)

    return cos_sim

def compute_word_similarity(word1, word2):
    '''
    Takes in two words, computes their embeddings and returns the cosine similarity
    '''
    emb_one = word2vec((word1)).squeeze()
    emb_two = word2vec((word2)).squeeze()
    return cosine_similarity(emb_one,emb_two).item()

# Define three words (one pair should be similar and one pair should be dissimilar) and compute their similarity
word1 = "man"
word2 = "woman"
word3 = "mango"
print(f"Similarity between '{word1}' and '{word2}': {compute_word_similarity(word1, word2)}")
print(f"Similarity between '{word1}' and '{word3}': {compute_word_similarity(word1, word3)}")

Similarity between 'man' and 'woman': 0.7608709335327148
Similarity between 'man' and 'mango': 0.12227462232112885


In [6]:
# Run this cell if you're done with the above section
del embedding_layer

## Sentence Classification Classification with Sentence Embeddings

Now let's move on to an actual application: classifying whether a tweet is about a real disaster or not. As you can imagine, this could be a valuable model when monitoring social media for disaster relief efforts.

Since we are using Sentence Embeddings, we want something that will take in a sequence of words and throw out a single fixed-size vector. For this task, we will make use of an LLM via the `gpt4all` library.

This library will allow us to generate pretrained embeddings for sentences, that we can use as **features** to feed to any classifier of our choice.

In [7]:
# Read in the data here
df = pd.read_csv("./disaster_tweets.csv")
df = df[["text", "target"]]

# Split the data
train, val = train_test_split(df, test_size=0.2, random_state=42)
print(train.shape, val.shape)

(6090, 2) (1523, 2)


Before jumping straight to Embeddings, since our data is sourced from the cesspool that is Twitter, we should probably do some cleaning. This can involve the removal of URLs, punctuation, numbers that don't provide any meaning, stopwords, and so on.'

In [ ]:
def lowercase(txt):
    
    return txt.lower()

def remove_punctuation(txt):
    
    return re.sub(r'[^\w\s]', '', txt)

def remove_stopwords(txt):
    
    stop_words = set(stopwords.words('english'))
    words = txt.split()
    return ' '.join([word for word in words if word not in stop_words])


def remove_numbers(txt):
    
    return re.sub(r'\d+', '', txt)

def remove_url(txt):
    
    return re.sub(r'http\S+', '', txt)

def normalize_sentence(txt):
    '''
    Aggregates all the above functions to normalize/clean a sentence
    '''
    
    txt = lowercase(txt)
    txt = remove_url(txt)
    txt = remove_punctuation(txt)
    txt = remove_numbers(txt)
    txt = remove_stopwords(txt)
    return txt

# Clean the sentences
train['text'] = train['text'].apply(normalize_sentence)
val['text'] = val['text'].apply(normalize_sentence)

# Filter sentences that are too short (less than 20ish characters)
train = train[train['text'].str.len() >= 20]
val = val[val['text'].str.len() >= 20]



Now for the fun part, creating our Embeddings!

We will be using the `gpt4all.Embed4All` class for this purpose. You can look up the documentation [here](https://docs.gpt4all.io/gpt4all_python_embedding.html#gpt4all.gpt4all.Embed4All.embed).

This functionality makes use of a model called [Sentence-BERT](https://arxiv.org/abs/1908.10084). This is a Transformer-based model that has been trained on a large corpus of text, and is able to generate high-quality Sentence Embeddings for us.

In [ ]:
# Generate embeddings for train and validation sentences (5 marks)

feature_extractor = Embed4All()

# Encode the train samples
# train_emb = feature_extractor.embed(train['text'])
# val_emb = feature_extractor.embed(val['text'])
# Encode the train sentences
train_emb = [feature_extractor.embed(sentence) for sentence in train['text']]

# Encode the validation sentences
val_emb = [feature_extractor.embed(sentence) for sentence in val['text']]

# Ready the labels
t_labels = train['target']
v_labels = val['target']


Now with our Embeddings ready, we can move on to the actual classification task.

We have the choice of using **any** classifier you wish. We can use a simple Logistic Regression model, get fancy with Support Vector Machines, or even use a Neural Network. The choice is ours.

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Initialize the SVM model
svm_model = SVC(probability=True)

# Convert and flatten the embeddings for training
train_emb_flat = [np.array(emb).flatten() for emb in train_emb]
val_emb_flat = [np.array(emb).flatten() for emb in val_emb]

# Train the SVM model
svm_model.fit(train_emb_flat, t_labels)

# Predict on the validation set
val_predictions = svm_model.predict(val_emb_flat)

# Calculate and print the accuracy
val_accuracy = accuracy_score(v_labels, val_predictions)
print(f"Validation Accuracy: {val_accuracy*100}")



Validation Accuracy: 82.73283480625425


In [ ]:
def predict(sentence, clf):
    '''
    Takes in a sentence and returns the predicted class along with the probability
    '''
    # Clean and encode the sentence
    cleaned_sentence = normalize_sentence(sentence)
    encoded_sentence = feature_extractor.embed(cleaned_sentence)
    encoded_sentence = np.array(encoded_sentence).reshape(1, -1)

    # Predict the class and probability
    prediction = clf.predict(encoded_sentence)
    probability = clf.predict_proba(encoded_sentence)
    return prediction[0], max(probability[0])
    
# Predict on a few of your own sentences
print(predict("I like thunderstorm and gloomy nights", svm_model))
print(predict("It is raining cats and dogs!", svm_model))
print(predict("During the 1960s the oryx a symbol of the Arabian Peninsula were annihilated by hunters.",svm_model))
print(predict("After death of Palestinian toddler in arson attack Israel cracks down on Jewish",svm_model))
print(predict("Mahira Khan's wedding and people's ugly commentary remind me how hard it is to be a woman and on top of that a divorcee who is financially independent. May the evil in you die, painfully!",svm_model))

(1, 0.6733840588143077)
(0, 0.6577374503394954)
(1, 0.8081076502988142)
(1, 0.9324382112820022)
(0, 0.9270193652165418)


Hopefully now you realize the power of Embeddings, and the usefulness of pretrained models.